In [1]:
!pip install findspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 9.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 13.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=6b6ba02c757c4415fd29f4f6dc2e2a564c3362098b4244304edb8c5396119e5c
  Stored in directory: /Users/lenson/Library/Caches/pip/wheels/95/13/41/f7f135ee114175605fb4f0a89e7389f3742aa6c1e1a5bcb657
Successfully built pyspark


In [1]:
import findspark

# Set up the necessary environment variables and adding the required dependencies to the Python path
findspark.init()
# Locate a local Spark installation.
findspark.find()

from pyspark.sql import SparkSession

# Initializes and returns a SparkSession object.
spark:SparkSession = SparkSession.builder \
    .master("local[8]") \
    .appName("assignment2") \
    .getOrCreate()

sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/24 12:02:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 53758)
Traceback (most recent call last):
  File "/Users/lenson/Documents/GitHub/NTU/SC4052-CloudComputing/Assignment2/.conda/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/lenson/Documents/GitHub/NTU/SC4052-CloudComputing/Assignment2/.conda/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/Users/lenson/Documents/GitHub/NTU/SC4052-CloudComputing/Assignment2/.conda/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/lenson/Documents/GitHub/NTU/SC4052-CloudComputing/Assignment2/.conda/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/Users/lenson/Documents/GitHub/NTU/SC4052-CloudComputing/Assignment2/.conda/lib/p

In [78]:
from pyspark.mllib.linalg.distributed import IndexedRow, IndexedRowMatrix
import numpy as np

b = 0.80
n = 4
matrix = [
    [0, 1/2, 0, 0], 
    [1/3, 0, 0, 1/2], 
    [1/3, 0, 1, 1/2], 
    [1/3, 1/2, 0, 0]]

In [73]:
initalRanks = sc.parallelize([[1/n]*n]).zipWithIndex()
initialB = sc.parallelize(np.diag([b]*n)).zipWithIndex()
initialJump = sc.parallelize([[(1-b)/n]*n]).zipWithIndex()
initialMatrix = sc.parallelize(matrix).zipWithIndex()

transitionMatrix =  IndexedRowMatrix( \
    initialMatrix \
    .map(lambda row: IndexedRow(row[1], row[0])) \
    ).toBlockMatrix()

ranks =  IndexedRowMatrix( \
    initalRanks \
    .map(lambda x: IndexedRow(x[1], x[0])) \
    ).toBlockMatrix()

b =  IndexedRowMatrix( \
    initialB \
    .map(lambda x: IndexedRow(x[1], x[0])) \
    ).toBlockMatrix()

jump = IndexedRowMatrix( \
    initialJump \
    .map(lambda x: IndexedRow(x[1], x[0])) \
    ).toBlockMatrix()

print(transitionMatrix.toLocalMatrix().toArray())
print(ranks.toLocalMatrix().toArray())
print(b.toLocalMatrix().toArray())
print(jump.toLocalMatrix().toArray())

[[0.         0.5        0.         0.        ]
 [0.33333333 0.         0.         0.5       ]
 [0.33333333 0.         1.         0.5       ]
 [0.33333333 0.5        0.         0.        ]]
[[0.25 0.25 0.25 0.25]]


[[0.8 0.  0.  0. ]
 [0.  0.8 0.  0. ]
 [0.  0.  0.8 0. ]
 [0.  0.  0.  0.8]]
[[0.05 0.05 0.05 0.05]]


In [76]:
ranksT = ranks.transpose() # vertical vector of ranks
jumpT = jump.transpose() # vertical vector of jump
numOfIterations = 0 # number of iterations
while True:
    numOfIterations += 1
    # newRanks = b * transitionMatrix ⋅ ranksT + jumpT
    newRanks = transitionMatrix.multiply(b).multiply(ranksT).add(jumpT)
    if np.equal(newRanks.toLocalMatrix().toArray(), ranksT.toLocalMatrix().toArray()).all():
        break
    ranksT = newRanks
# print the result
print(ranksT.toLocalMatrix().toArray(), numOfIterations)

[[0.10135135]
 [0.12837838]
 [0.64189189]
 [0.12837838]] 69
